In [1]:
import pandas as pd
import numpy as np
import re
import requests
import time
import validators
import datetime
from requests import get
from bs4 import BeautifulSoup
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from time import sleep 
from datetime import datetime
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from urllib.parse import urlsplit

In [2]:
url_1 = 'https://www.artistsandfleas.com/williamsburg/'
url_2 = 'https://www.artistsandfleas.com/chelsea/'

urls_to_try = ["","/contact","/contact-us","/about"]

In [3]:
def get_them_emails(url):

    response = None
    contact_email = None
    parts = []
    df_list = []
    
    try:
        response = requests.get(url, timeout = 10)
    except:
        pass
    else:
        parts = urlsplit(url)

    if response != None:
        for i in range(len(urls_to_try)):
            try:
                response = requests.get("{0.scheme}://{0.netloc}".format(parts)+ urls_to_try[i])
                print("{0.scheme}://{0.netloc}".format(parts) + urls_to_try[i])
            except:
                pass
            try:
                contact_email = re.findall(r'([a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+)',response.text)
            except:
                pass
            try:
                df_list.append(contact_email)
            except:
                pass
    return df_list

def drop_duplicates(lst):
    res = []
    for i in lst:
        if i not in res:
            res.append(i)
    return res

def flatten(xss):
    return [x for xs in xss for x in xs]


def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


blacklist = ['png','jpg', 'example', 'exemple', 'logo','Logo','LOGO','j.doe']

def clean_list(lst):
    i = 0
    while i < len(lst):
        if (blacklist[0] in lst[i])\
        or (blacklist[1] in lst[i])\
        or (blacklist[2] in lst[i])\
        or (blacklist[3] in lst[i])\
        or (blacklist[4] in lst[i])\
        or (blacklist[5] in lst[i])\
        or (blacklist[6] in lst[i])\
        or (blacklist[7] in lst[i])\
        or has_numbers(lst[i]):
            lst.remove(lst[i])
            i -= 1
        i += 1
    return lst

In [4]:
reqs = requests.get(url_1)
soup = BeautifulSoup(reqs.text, 'html.parser')

urls = []
names = []
for link in soup.find_all('a'):
    urls.append(link.get('href'))
    names.append(link.get_text())

reqs = requests.get(url_2)
soup = BeautifulSoup(reqs.text, 'html.parser')
 
for link in soup.find_all('a'):
    urls.append(link.get('href'))
    names.append(link.get_text())

In [5]:
d = {'name':names,'url':urls}
df = pd.DataFrame(d)

df1 = df[((-df['url'].str.contains('artist',na=False))
    &(-df['url'].str.contains('instagram',na=False))
    &(-df['url'].str.contains('bike',na=False))
    &(-df['url'].str.contains('mta',na=False))
    &(-df['url'].str.contains('ferry',na=False))
    &(-df['url'].str.contains('find-us',na=False))
    &(-df['url'].str.contains('etsy',na=False))
    &(-df['url'].str.contains('restaurant',na=False))
    &(-df['url'].str.contains('google',na=False))
    &(-df['url'].str.contains('facebook',na=False))
    &(-df['name'].str.contains('Visit',na=False))
   )]

df1

,name,url
12,Dani and the Rings,https://www.daniandtherings.com/
14,i.e. Spa Indulgences,https://ie-spa-indulgences.com/
15,GrandMother Goods,https://www.gmother.com/
16,Keila Jewelry,https://keilajewelry.com/
24,CTBT Studios (Contribute Studios),http://ctbtstudio.com/
25,Marina Pecoraro Jewelry,http://marinapecorarojewelry.com/
27,D.Webb Designs,https://dwebbdesigns.com/
28,Only You Skincare,http://www.onlyyouskincare.com/
67,Dani and the Rings,https://www.daniandtherings.com/
68,Jewels By Atlantis,https://www.jewelsbyatlantis.com/


In [6]:
df1['email'] = df1['url'].apply(get_them_emails)

https://www.daniandtherings.com
https://www.daniandtherings.com/contact
https://www.daniandtherings.com/contact-us
https://www.daniandtherings.com/about
https://ie-spa-indulgences.com
https://ie-spa-indulgences.com/contact
https://ie-spa-indulgences.com/contact-us
https://ie-spa-indulgences.com/about
https://www.gmother.com
https://www.gmother.com/contact
https://www.gmother.com/contact-us
https://www.gmother.com/about
https://keilajewelry.com
https://keilajewelry.com/contact
https://keilajewelry.com/contact-us
https://keilajewelry.com/about
http://ctbtstudio.com
http://ctbtstudio.com/contact
http://ctbtstudio.com/contact-us
http://ctbtstudio.com/about
http://marinapecorarojewelry.com
http://marinapecorarojewelry.com/contact
http://marinapecorarojewelry.com/contact-us
http://marinapecorarojewelry.com/about
https://dwebbdesigns.com
https://dwebbdesigns.com/contact
https://dwebbdesigns.com/contact-us
https://dwebbdesigns.com/about
http://www.onlyyouskincare.com
http://www.onlyyouskincare

C:\Users\Usuario\AppData\Local\Temp/ipykernel_42232/4139400919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['email'] = df1['url'].apply(get_them_emails)


In [7]:
df1['email_clean'] = df1.email
df1['email_clean'] = df1['email_clean'].apply(flatten)
df1['email_clean'] = df1['email_clean'].apply(clean_list)
df1['email_clean'] = df1['email_clean'].apply(drop_duplicates)
df1

C:\Users\Usuario\AppData\Local\Temp/ipykernel_42232/3185941800.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['email_clean'] = df1.email
C:\Users\Usuario\AppData\Local\Temp/ipykernel_42232/3185941800.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['email_clean'] = df1['email_clean'].apply(flatten)
C:\Users\Usuario\AppData\Local\Temp/ipykernel_42232/3185941800.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,name,url,email,email_clean
12,Dani and the Rings,https://www.daniandtherings.com/,"[[Angelos@daniandtherings.com], [Angelos@dania...",[Angelos@daniandtherings.com]
14,i.e. Spa Indulgences,https://ie-spa-indulgences.com/,"[[john@example.com], [john@example.com], [john...",[]
15,GrandMother Goods,https://www.gmother.com/,"[[grandmothergoods@gmail.com, grandmothergoods...",[grandmothergoods@gmail.com]
16,Keila Jewelry,https://keilajewelry.com/,"[[fullsizeoutput_8bb_110x@2x.jpg, image_7b1a4f...",[nyc@keilajewelry.com]
24,CTBT Studios (Contribute Studios),http://ctbtstudio.com/,"[[core-js-bundle@3.2.1, focus-within-polyfill@...",[hello@ctbtstudio.com]
25,Marina Pecoraro Jewelry,http://marinapecorarojewelry.com/,"[[Marinamatia@hotmail.com], [Marinamatia@hotma...",[Marinamatia@hotmail.com]
27,D.Webb Designs,https://dwebbdesigns.com/,"[[], [], [], []]",[]
28,Only You Skincare,http://www.onlyyouskincare.com/,"[[example@mail.com, user@example.com, example@...",[]
67,Dani and the Rings,https://www.daniandtherings.com/,"[[], [Angelos@daniandtherings.com], [Angelos@d...",[Angelos@daniandtherings.com]
68,Jewels By Atlantis,https://www.jewelsbyatlantis.com/,"[[], [], [user@example.com], []]",[]


In [8]:
df2 = df[df['url'].str.contains('instagram',na=False)]
df2

,name,url
13,Eat Records,https://www.instagram.com/eatrecordsnyc/?hl=en
17,The Curious Desk & Arthurious,https://www.instagram.com/thecuriousdesk/
18,Beauty Crisis,https://www.instagram.com/beautycrisisvintagej...
19,Queer Candle Co.,https://www.instagram.com/queercandleco/
20,Musu Gift Shop,https://www.instagram.com/musugiftshop/
21,Vamos Al Grano,https://www.instagram.com/vamosalgrano.bklyn/
22,Fork Art,https://www.instagram.com/myforkart/
23,Good Morning Stella INC.,https://instagram.com/gm_stella
29,PopTrain Records,https://www.instagram.com/poptrainrecords/
52,\n\n\n,https://instagram.com/artistsandfleas


In [9]:
df2['url_insta'] =  np.nan
df2.head()

C:\Users\Usuario\AppData\Local\Temp/ipykernel_42232/2387038659.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['url_insta'] =  np.nan


,name,url,url_insta
13,Eat Records,https://www.instagram.com/eatrecordsnyc/?hl=en,NaN
17,The Curious Desk & Arthurious,https://www.instagram.com/thecuriousdesk/,NaN
18,Beauty Crisis,https://www.instagram.com/beautycrisisvintagej...,NaN
19,Queer Candle Co.,https://www.instagram.com/queercandleco/,NaN
20,Musu Gift Shop,https://www.instagram.com/musugiftshop/,NaN


In [10]:
for i in range(len(df2)):
    reqs = requests.get(df2.url.iloc[i])
    soup = BeautifulSoup(reqs.text, 'html.parser')
    string_1 = str(soup)[str(soup).find("external_url")+17:str(soup).find("external_url")+100]
    string_2 = string_1[:string_1.find("external_url")]
    regex = re.compile('[\\\,]') #etc
    string_3 = regex.sub('', string_2).replace('"','')
    print(string_3)
    df2.url_insta.iloc[i] = string_3

https://eatrecords.bigcartel.com/


C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


http://thecuriousdesk.com/


C:\Users\Usuario\anaconda3\lib\site-packages\pandas\core\indexing.py:1965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj._check_is_chained_assignment_possible()


http://www.beautycrisisvintage.com/
https://queercandleco.com/collections/thrifted-candle-containerse
ll
ll
http://myforkart.com/
http://www.gmstella.com/
https://www.discogs.com/seller/PopTrainRecords/profileexternal
https://linktr.ee/artistsandfleas_
http://www.curatedbasics.com/
http://www.mikelindwasserphotography.com/
http://www.pasteusa.com/
http://thecuriousdesk.com/
http://www.pinkypilots.com/
https://www.patentofheart.com/shop
ll
https://linktr.ee/artistsandfleas_


In [11]:
df2 = df2[df2['url_insta'].str.len()>5]

In [12]:
df2['email'] = df2['url_insta'].apply(get_them_emails)

https://eatrecords.bigcartel.com
https://eatrecords.bigcartel.com/contact
https://eatrecords.bigcartel.com/contact-us
https://eatrecords.bigcartel.com/about
http://thecuriousdesk.com
http://thecuriousdesk.com/contact
http://thecuriousdesk.com/contact-us
http://thecuriousdesk.com/about
https://queercandleco.com
https://queercandleco.com/contact
https://queercandleco.com/contact-us
https://queercandleco.com/about
http://myforkart.com
http://myforkart.com/contact
http://myforkart.com/contact-us
http://myforkart.com/about
http://www.gmstella.com
http://www.gmstella.com/contact
http://www.gmstella.com/contact-us
http://www.gmstella.com/about
https://www.discogs.com
https://www.discogs.com/contact
https://www.discogs.com/contact-us
https://www.discogs.com/about
https://linktr.ee
https://linktr.ee/contact
https://linktr.ee/contact-us
https://linktr.ee/about
http://www.curatedbasics.com
http://www.curatedbasics.com/contact
http://www.curatedbasics.com/contact-us
http://www.curatedbasics.com/ab

C:\Users\Usuario\AppData\Local\Temp/ipykernel_42232/3558145461.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['email'] = df2['url_insta'].apply(get_them_emails)


In [13]:
df2['email_clean'] = df2.email
df2['email_clean'] = df2['email_clean'].apply(flatten)
df2['email_clean'] = df2['email_clean'].apply(clean_list)
df2['email_clean'] = df2['email_clean'].apply(drop_duplicates)
df2

C:\Users\Usuario\AppData\Local\Temp/ipykernel_42232/1315665483.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['email_clean'] = df2.email
C:\Users\Usuario\AppData\Local\Temp/ipykernel_42232/1315665483.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['email_clean'] = df2['email_clean'].apply(flatten)
C:\Users\Usuario\AppData\Local\Temp/ipykernel_42232/1315665483.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,name,url,url_insta,email,email_clean
13,Eat Records,https://www.instagram.com/eatrecordsnyc/?hl=en,https://eatrecords.bigcartel.com/,"[[], [], [], []]",[]
17,The Curious Desk & Arthurious,https://www.instagram.com/thecuriousdesk/,http://thecuriousdesk.com/,"[[], [], [], []]",[]
18,Beauty Crisis,https://www.instagram.com/beautycrisisvintagej...,http://www.beautycrisisvintage.com/,[],[]
19,Queer Candle Co.,https://www.instagram.com/queercandleco/,https://queercandleco.com/collections/thrifted...,"[[Summer_2022_Release_1280x@2x.jpg, P3110038_2...",[info@queercandleco.com]
22,Fork Art,https://www.instagram.com/myforkart/,http://myforkart.com/,"[[core-js-bundle@3.2.1, focus-within-polyfill@...",[]
23,Good Morning Stella INC.,https://instagram.com/gm_stella,http://www.gmstella.com/,"[[], [], [], []]",[]
29,PopTrain Records,https://www.instagram.com/poptrainrecords/,https://www.discogs.com/seller/PopTrainRecords...,"[[387a0c015e78429ca256c602993e2b92@sentry.io],...",[]
52,\n\n\n,https://instagram.com/artistsandfleas,https://linktr.ee/artistsandfleas_,"[[], [], [], []]",[]
69,Curated Basics,https://www.instagram.com/curatedbasics/,http://www.curatedbasics.com/,"[[info@curatedbasics.com, email@example.com, e...",[info@curatedbasics.com]
70,Mike Lindwasser Photography,https://www.instagram.com/mikelindwasserphotog...,http://www.mikelindwasserphotography.com/,"[[], [], [], []]",[]


In [21]:
df1.drop(columns=['url','email'])
df2.drop(columns=['url','url_insta','email'])

,name,email_clean
13,Eat Records,[]
17,The Curious Desk & Arthurious,[]
18,Beauty Crisis,[]
19,Queer Candle Co.,[info@queercandleco.com]
22,Fork Art,[]
23,Good Morning Stella INC.,[]
29,PopTrain Records,[]
52,\n\n\n,[]
69,Curated Basics,[info@curatedbasics.com]
70,Mike Lindwasser Photography,[]


In [34]:
frames = [df1.drop(columns=['url','email']),df2.drop(columns=['url','url_insta','email'])]
df_email = pd.concat(frames)
df_email.reset_index(drop=True, inplace=True)

df_email['email'] = ""
for i in range(len(df_email)-1):
    if not not df_email.iloc[i,1]:
        df_email.email[i] = df_email.iloc[i,1][0]

df_email['source'] = "Artists and Fleas"
df_email = df_email.drop(columns=['email_clean'])

df_email.email.replace('', np.nan, inplace=True)

df_email.dropna(subset=['email'], inplace=True)
df_email.reset_index(drop=True, inplace=True)

df_email = df_email.drop_duplicates(subset=['email'], keep='first', inplace=False, ignore_index=False)
df_email

,name,email,source
0,Dani and the Rings,Angelos@daniandtherings.com,Artists and Fleas
1,GrandMother Goods,grandmothergoods@gmail.com,Artists and Fleas
2,Keila Jewelry,nyc@keilajewelry.com,Artists and Fleas
3,CTBT Studios (Contribute Studios),hello@ctbtstudio.com,Artists and Fleas
4,Marina Pecoraro Jewelry,Marinamatia@hotmail.com,Artists and Fleas
6,Chacana,chacananyc@gmail.com,Artists and Fleas
7,Brooklyn Charm,info@brooklyncharm.com,Artists and Fleas
8,Live By The Sword,help@livebytheswordtattoo.com,Artists and Fleas
9,"eMCee Apparel, L.L.C.",Sales@eMCeeKicks.com,Artists and Fleas
10,"Spoolish, LLC",emily@spoolish.com,Artists and Fleas


In [35]:
df_email.to_csv('22-07-28 NY Brand Emails.csv',index=False)